In [1]:
### Module 0: Importing Libraries
import pandas as pd
import numpy as np
import pprint
import os 
from time import time 
from dotenv import load_dotenv
import json
import argparse
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from openai import OpenAI
import certifi

## Importing VE libraries
from utils import *
from dataset_utils import read_wikiqa_data
from prompt_helper import get_joint_prompt_helper, normalize_prediction
from dataset_utils import read_wikiqa_data, wiki_evaluation
# Override bad SSL_CERT_FILE if set
os.environ["SSL_CERT_FILE"] = certifi.where()

In [2]:
import os
import argparse
from tqdm import tqdm
from utils import *
from dataset_utils import read_wikiqa_data
# from comp_utils import model_embeddings
import sklearn
from sentence_transformers import SentenceTransformer
import consistency
import pandas as pd
import torch
import verifying_questions
from prompt_helper import get_joint_prompt_helper
from matplotlib import pyplot as plt
import numpy as np
from nltk import tokenize
import logging

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from relevant_context import _parse_args as relevant_parse_args, test_few_shot_manual_prediction as relevant_context_test_few_shot_manual_prediction

In [4]:
args = relevant_parse_args()


### Executing Code

In [5]:
print("Running prediction")
train_set = read_wikiqa_data(f"data/train_subset.json", manual_annotation_style=args.style)
train_set = train_set[args.train_slice:(args.train_slice + args.num_shot)]
print('len(train_set): ', len(train_set))
dev_set = read_wikiqa_data(f"data/dev_sampled.json")
dev_set = dev_set[args.dev_slice:(args.num_dev)]

# finished consistency and processs
predictions = read_full(args, consistency)
new_predictions, cons = [], []
for i, p in enumerate(tqdm(predictions, total=len(predictions), desc="Verifying")):
    ex = dev_set[i]
    con, new_p = consistency.post_process_consistency(ex, p, args)
    cons.append(con)
    new_predictions.append(new_p)
predictions = new_predictions 

Running prediction
0 not found
len(train_set):  5
7 not found


Verifying: 100%|██████████| 10/10 [00:00<00:00, 17734.90it/s]


In [6]:
len(predictions)

10

In [7]:
verifying_qs = read_full(args, verifying_questions, slice = False)
len(verifying_qs)

3

In [8]:
contexts = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

if 'drqa' in args.retrieval:
    drqa_contexts = read_json_txt(args.drqa_path)
if args.retrieval == 'google':
    with open(args.google_path, 'rb') as f:
        google_contexts = pickle.load(f)

broken = False
includes_before = []
includes_after = []
numbers = []
num_supps = []
num_keyword_overlaps = []

In [9]:
for i, p in enumerate(tqdm(predictions, total=len(predictions), desc="Verifying")):
    cont = []
    if p['consistency'] < args.consistency_threshold: # only predict again when lower consistency
        ex = dev_set[i]
        gt_cont = ex['supp_pars']
        sentences = rationale_tokenize(p['rationale'])
        id = ex['id']

Verifying: 100%|██████████| 10/10 [00:00<00:00, 5417.60it/s]


In [10]:
cont, id, sentences

([],
 '9c4a7a4f090c11ebbdadac1f6bf848b6',
 ['The Statement (Film) is a film from France. ',
  'A Self Made Hero is also a film from France.'])

In [11]:
vq = [c for c in verifying_qs if c['id']==id][0]['verifying_questions']
all_pars_text = []
all_pars = []
vq

['What country is the film A Self Made Hero from?',
 'What country is the film A Self Made Hero from?']

In [12]:
sent = vq[0]
sent

'What country is the film A Self Made Hero from?'

In [39]:
sent = "Where did Ingmar Bergman, the director of the film Temptation (1959 Film), die?"

In [40]:
start = time()
pars_text = get_texts_to_rationale_wikipedia(sent , False)
end = time()

A 0.008889198303222656
B 1.5065557956695557
C 0.8164820671081543
D 8.702278137207031e-05
E 0.00012493133544921875
F 0.4361579418182373
G 46.5465452671051
H 28.366061687469482


In [15]:
pars_text

['A Self-Made Hero (French: Un héros très discret) is a 1996 French film directed by Jacques Audiard.',
 'It is based on the novel by Jean-François Deniau.',
 'Synopsis\nThe film is presented in the style of a historical documentary, with an elderly Dehousse narrating the events of his life, interspersed with interviews with other characters and historians.',
 "Albert Dehousse has grown up on heroic novels, unfortunately his life isn't quite so exciting.",
 'Albert lives in a village in northern France with his mother, who lives in memory of her husband, who she claims died a hero in the First World War.',
 'SelfMadeHero is an independent publishing house which specialises in adapting works of literature, as well as producing ground-breaking original fiction in the graphic novel medium.',
 "SelfMadeHero's books are distributed in the UK by Abrams & Chronicle Books and in the U.S.",
 'by Abrams Books.',
 'History\nSelfMadeHero was founded in February 2007 by Emma Hayley, and launched wi

In [16]:
len(pars_text)

50

In [17]:
pars_text = list(dict.fromkeys(pars_text)) #remove potential duplicates
all_pars_text += pars_text

In [18]:
len(pars_text)

45

In [21]:
def model_embeddings(sentence, model):
    embedding = model.encode([sentence])
    return embedding[0] #should return an array of shape 384

In [23]:
sen_embeds = [model_embeddings(sent, embedding_model)]
par_embeds = [model_embeddings(s, embedding_model) for s in pars_text]



In [26]:
pars = sklearn.metrics.pairwise.pairwise_distances(sen_embeds, par_embeds)


In [27]:
pars

array([[5.5609794, 7.550731 , 6.408272 , 7.193566 , 6.89299  , 7.165553 ,
        7.1266055, 8.92165  , 5.9714594, 7.9653397, 7.3592205, 8.020057 ,
        8.087286 , 7.9294963, 7.7201314, 7.4307632, 6.5333943, 6.599609 ,
        6.2533507, 5.968159 , 8.069149 , 8.218054 , 8.522786 , 7.7088785,
        8.208936 , 7.1057997, 7.1435347, 7.2951593, 8.40378  , 9.074446 ,
        6.3123426, 6.849242 , 6.6262903, 6.314308 , 7.3572206, 6.2789574,
        6.527658 , 6.409714 , 7.453066 , 7.285027 , 5.9770036, 6.87745  ,
        6.276412 , 6.8885446, 6.1412134]], dtype=float32)

In [28]:
args.topk

3

In [29]:
pars = pars.argsort(axis = 1)[0][:args.topk]

In [30]:
pars

array([ 0, 19,  8])

In [31]:
pars = [pars_text[i] for i in pars]
cont.append(pars)
all_pars += pars

In [32]:
all_pars

['A Self-Made Hero (French: Un héros très discret) is a 1996 French film directed by Jacques Audiard.',
 'His drama The Beat That My Heart Skipped (2005) was seen as his breakout film, earning him a BAFTA and his first César trifecta, followed by a second for the prison crime drama A Prophet (2009), which earned a nomination for the Academy Award for Best Foreign Language Film.',
 'History\nSelfMadeHero was founded in February 2007 by Emma Hayley, and launched with two lines: Manga Shakespeare, featuring works based on the Bard but with different settings – mainly Japan in the past and future, and Eye Classics, which are adaptations of great classic works, such as those of Poe and Kafka.']

### Breaking get_texts_to_rationale_wikipedia function

In [33]:
sent = vq[0]
sent

'What country is the film A Self Made Hero from?'

In [ ]:
# # pars_text = get_texts_to_rationale_wikipedia(vq[j], False)

# def get_texts_to_rationale_wikipedia(rationale, no_links):
#     t1 = time()
#     valid_ents = find_ents(rationale)
#     t2 =time()
#     print("A", t2-t1)
#     # find pages
#     pages_to_ents = relevant_pages_to_ents(valid_ents, topk = 5)
#     t3 = time()
#     print("B", t3-t2)
#     pages_to_rationale = relevant_pages_to_rationale(rationale, topk = 5)
#     t4 =time()
#     print("C", t4-t3)
#     page_titles = pages_to_ents + pages_to_rationale
#     t5 =time()
#     print("D", t5- t4)
#     pages = get_wiki_objs(page_titles)
#     t6 = time()
#     print("E", t6-t5)
#     new_pages = []
#     for i, pt in enumerate(page_titles):
#         if 'disambiguation' in pt:
#             new_pages += get_linked_pages([pages[i]], topk = -1)
#         else:
#             new_pages += [pages[i]]
#     t7 =time()
#     print("F", t7-t6)
#     pages = new_pages
#     if not no_links:
#         # add linked pages
#         pages + get_linked_pages(pages, topk = 5)
#     # get rid of duplicates
#     t8 = time()
#     print("G", t8-t7)
#     pages = list(dict.fromkeys(pages))
#     # get texts
#     texts = get_texts_to_pages(pages, topk = 5)
#     t9 =time()
#     print("H", t9-t8)
#     return texts

In [41]:
sent = "Where did Ingmar Bergman, the director of the film Temptation (1959 Film), die?"

In [42]:
rationale = sent

In [46]:

valid_ents = find_ents(rationale)
valid_ents

['Ingmar Bergman', 'Temptation (1959 Film']

In [47]:
# can be reduced to 3?
pages_to_ents = relevant_pages_to_ents(valid_ents, topk = 5)
pages_to_ents

['Ingmar Bergman',
 'Ingmar Bergman filmography',
 'The Seduction of Ingmar Bergman',
 'Gun Bergman',
 'Erik Bergman (Lutheran minister)',
 'Temptation (1959 film)',
 'Temptation (disambiguation)',
 'Temptation Island',
 'Temptation Harbour',
 'The Temptations']

In [48]:
# Giving out very irrelevant topics, should be reduced to 2-3 max
pages_to_rationale = relevant_pages_to_rationale(rationale, topk = 5)
pages_to_rationale

['List of film director–composer collaborations',
 'Andrei Tarkovsky',
 'Martin Scorsese',
 'List of film director and cinematographer collaborations',
 'List of directorial debuts']

In [49]:
page_titles = pages_to_ents + pages_to_rationale
page_titles

['Ingmar Bergman',
 'Ingmar Bergman filmography',
 'The Seduction of Ingmar Bergman',
 'Gun Bergman',
 'Erik Bergman (Lutheran minister)',
 'Temptation (1959 film)',
 'Temptation (disambiguation)',
 'Temptation Island',
 'Temptation Harbour',
 'The Temptations',
 'List of film director–composer collaborations',
 'Andrei Tarkovsky',
 'Martin Scorsese',
 'List of film director and cinematographer collaborations',
 'List of directorial debuts']

In [50]:
pages = get_wiki_objs(page_titles)
pages

[Ingmar Bergman (lang: en, variant: None, id: ??, ns: 0),
 Ingmar Bergman filmography (lang: en, variant: None, id: ??, ns: 0),
 The Seduction of Ingmar Bergman (lang: en, variant: None, id: ??, ns: 0),
 Gun Bergman (lang: en, variant: None, id: ??, ns: 0),
 Erik Bergman (Lutheran minister) (lang: en, variant: None, id: ??, ns: 0),
 Temptation (1959 film) (lang: en, variant: None, id: ??, ns: 0),
 Temptation (disambiguation) (lang: en, variant: None, id: ??, ns: 0),
 Temptation Island (lang: en, variant: None, id: ??, ns: 0),
 Temptation Harbour (lang: en, variant: None, id: ??, ns: 0),
 The Temptations (lang: en, variant: None, id: ??, ns: 0),
 List of film director–composer collaborations (lang: en, variant: None, id: ??, ns: 0),
 Andrei Tarkovsky (lang: en, variant: None, id: ??, ns: 0),
 Martin Scorsese (lang: en, variant: None, id: ??, ns: 0),
 List of film director and cinematographer collaborations (lang: en, variant: None, id: ??, ns: 0),
 List of directorial debuts (lang: en, 

In [51]:
new_pages = []
for i, pt in enumerate(page_titles):
    if 'disambiguation' in pt:
        new_pages += get_linked_pages([pages[i]], topk = -1)
    else:
        new_pages += [pages[i]]

In [52]:
new_pages

[Ingmar Bergman (lang: en, variant: None, id: ??, ns: 0),
 Ingmar Bergman filmography (lang: en, variant: None, id: ??, ns: 0),
 The Seduction of Ingmar Bergman (lang: en, variant: None, id: ??, ns: 0),
 Gun Bergman (lang: en, variant: None, id: ??, ns: 0),
 Erik Bergman (Lutheran minister) (lang: en, variant: None, id: ??, ns: 0),
 Temptation (1959 film) (lang: en, variant: None, id: ??, ns: 0),
 Around the World in a Day (lang: en, variant: None, id: ??, ns: 0),
 Franks Wild Years (lang: en, variant: None, id: ??, ns: 0),
 Get Happy!! (Elvis Costello album) (lang: en, variant: None, id: ??, ns: 0),
 Get Squared (lang: en, variant: None, id: ??, ns: 0),
 Higher and Higher: The Best of Heaven 17 (lang: en, variant: None, id: ??, ns: 0),
 IV (Godsmack album) (lang: en, variant: None, id: ??, ns: 0),
 List of Code Lyoko episodes (lang: en, variant: None, id: ??, ns: 0),
 List of Italian films of 1968 (lang: en, variant: None, id: ??, ns: 0),
 List of The Price Is Right pricing games (lan

In [53]:
len(new_pages)

65

In [54]:
def get_linked_pages(wiki_pages, topk = 5):
    linked_ents = []
    for wp in wiki_pages:
        linked_ents += list(wp.links.values())
        if topk != -1:
            linked_ents = linked_ents[:topk]
    return linked_ents


In [56]:
x = page_titles[6]

In [62]:
type(pages[6])

wikipediaapi.WikipediaPage

In [59]:
x = [pages[6]]

In [65]:
get_linked_pages(x, topk= 3)

[Around the World in a Day (lang: en, variant: None, id: ??, ns: 0),
 Franks Wild Years (lang: en, variant: None, id: ??, ns: 0),
 Get Happy!! (Elvis Costello album) (lang: en, variant: None, id: ??, ns: 0)]

In [67]:
no_links = False

In [72]:
len(new_pages)

65

In [68]:
pages = new_pages
if not no_links:
    # add linked pages
    pages + get_linked_pages(pages, topk = 5)

In [69]:
pages

[Ingmar Bergman (lang: en, variant: None, id: 14626, ns: 0),
 Ingmar Bergman filmography (lang: en, variant: None, id: 13075438, ns: 0),
 The Seduction of Ingmar Bergman (lang: en, variant: None, id: 24304987, ns: 0),
 Gun Bergman (lang: en, variant: None, id: 68025669, ns: 0),
 Erik Bergman (Lutheran minister) (lang: en, variant: None, id: 12544302, ns: 0),
 Temptation (1959 film) (lang: en, variant: None, id: 49072114, ns: 0),
 Around the World in a Day (lang: en, variant: None, id: 1922114, ns: 0),
 Franks Wild Years (lang: en, variant: None, id: 589981, ns: 0),
 Get Happy!! (Elvis Costello album) (lang: en, variant: None, id: 1344544, ns: 0),
 Get Squared (lang: en, variant: None, id: 5247678, ns: 0),
 Higher and Higher: The Best of Heaven 17 (lang: en, variant: None, id: 62723611, ns: 0),
 IV (Godsmack album) (lang: en, variant: None, id: 4131804, ns: 0),
 List of Code Lyoko episodes (lang: en, variant: None, id: 4699160, ns: 0),
 List of Italian films of 1968 (lang: en, variant: 

In [70]:
len(pages)

65

In [73]:
new_pages

[Ingmar Bergman (lang: en, variant: None, id: 14626, ns: 0),
 Ingmar Bergman filmography (lang: en, variant: None, id: 13075438, ns: 0),
 The Seduction of Ingmar Bergman (lang: en, variant: None, id: 24304987, ns: 0),
 Gun Bergman (lang: en, variant: None, id: 68025669, ns: 0),
 Erik Bergman (Lutheran minister) (lang: en, variant: None, id: 12544302, ns: 0),
 Temptation (1959 film) (lang: en, variant: None, id: 49072114, ns: 0),
 Around the World in a Day (lang: en, variant: None, id: 1922114, ns: 0),
 Franks Wild Years (lang: en, variant: None, id: 589981, ns: 0),
 Get Happy!! (Elvis Costello album) (lang: en, variant: None, id: 1344544, ns: 0),
 Get Squared (lang: en, variant: None, id: 5247678, ns: 0),
 Higher and Higher: The Best of Heaven 17 (lang: en, variant: None, id: 62723611, ns: 0),
 IV (Godsmack album) (lang: en, variant: None, id: 4131804, ns: 0),
 List of Code Lyoko episodes (lang: en, variant: None, id: 4699160, ns: 0),
 List of Italian films of 1968 (lang: en, variant: 

### Rerunning optimized code

In [3]:
sent = "Where did Ingmar Bergman, the director of the film Temptation (1959 Film), die?"
start = time()
pars_text = get_texts_to_rationale_wikipedia(sent , False)
end = time()

A 0.00938105583190918
B 1.451786994934082
C 1.0471141338348389
D 0.0002658367156982422
E 0.0001270771026611328
F 0.4334688186645508
G 0.0002951622009277344
H 7.196820020675659
